In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

%cd ..
!ls
from core import *

/Users/trippgordon/Desktop/Code/sonic/NoisyGAN
Data        README.md   core        models      submit.sh
Notebooks   __init__.py depricated  notes_ideas train.py


### Generator

input shape (100)

upsample into (f0_scaled, ld_scaled) (2x1000) 

run through decoder & processor group

output shape (64000)

In [2]:
gen = Generator(latent_dim=100)
fake_sample = gen.generate(label=0)
describeSample(fake_sample)
play(fake_sample['audio'])

# params:  1276365

audio: (64000,)
f0_hz: (1000,)
loudness_db: (1000,)
label: (1,)


### Get alternating batches

Maybe getting combined batches would be better...

In [3]:
def combined_sample_generator(gen, dataset, batch_size=8):
    half_batch = int(batch_size/2)
    shapes = {"f0_hz": (1000,),
                    "loudness_db": (1000,),
                    "audio": (64000,),
                    "label": (1,)}
    
    i = -1
    while True:
        i+=1
        logging.info("generating... batch "+str(i))
        generated = gen.generate_batch(label=0,batch_size=half_batch)
        real = next(iter(dataset.data_provider.get_batch(half_batch, shuffle=True, repeats=-1)))
        batch = {}
        for key in shapes:
            batch[key] = tf.concat([generated[key],real[key]],axis=0)

        yield batch
             
def real_sample_generator(dataset, batch_size=8):
    i = -1
    while True:
        i+=1
        real = next(iter(dataset.data_provider.get_batch(batch_size, shuffle=True, repeats=-1)))
        yield real
        
def fake_sample_generator(gen, batch_size=8):
    shapes = {"f0_hz": (1000,),
                    "loudness_db": (1000,),
                    "audio": (64000,),
                    "label": (1,)}
    
    i = -1
    while True:
        i+=1
        logging.info("generating... batch "+str(i))
        generated = gen.generate_batch(label=0,batch_size=batch_size)
        batch = {}
        for key in shapes:
            batch[key] = generated[key]

        yield batch

In [4]:
# dataset_iter = iter(real_sample_generator(dataset))
# dataset_iter = iter(fake_sample_generator(gen))

# batch = next(dataset_iter)
# describeSample(batch)
# batch['label'], batch['f0_hz'][0][0]

### Discriminator

input shape (64000)

encode into latent space (6 x 1000)

- f0_scaled (1 x 1000)
- ld_scaled (1 x 1000)
- z (4 x 1000)

output shape (1)

In [5]:
def train_discriminator_loop(discriminator, opt, dataset_iter, iters=10):
    # unfreeze weights
    discriminator.trainable = True
    
    for i in range(iters): #todo: shuffle batch around!
        batch = next(dataset_iter)
        # train_step
        grad_clip_norm = 3.0
        with tf.GradientTape() as tape:
            pred = discriminator(batch,training=True)
            print(pred,batch['label'])
            total_loss = tf.reduce_sum(discriminator.losses)
            print("Step " + str(i) + " Loss: " + str(total_loss))
        grads = tape.gradient(total_loss, discriminator.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, grad_clip_norm)
        opt.apply_gradients(zip(grads, discriminator.trainable_variables))

In [6]:
batch_size = 8

gen = Generator(latent_dim=100)
dataset = DDSP_DATASET('./Data/piano/piano30s.tfrecord')
combined_iter = iter(combined_sample_generator(gen, dataset, batch_size=8))

# discriminate
discriminator = Discriminator(batch_size=batch_size)
opt = tf.keras.optimizers.Adam()

# params:  1276365



In [7]:
# train
train_discriminator_loop(discriminator, opt, combined_iter, iters=10)

INFO:absl:generating... batch 0


tf.Tensor(
[[0.6300374 ]
 [0.35300586]
 [0.7031488 ]
 [0.47912988]
 [0.73186034]
 [0.701914  ]
 [0.6286642 ]
 [0.35625196]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Step 0 Loss: tf.Tensor(5.458934, shape=(), dtype=float32)


INFO:absl:generating... batch 1


tf.Tensor(
[[0.225557  ]
 [0.2151773 ]
 [0.15888803]
 [0.1716636 ]
 [0.41531444]
 [0.994183  ]
 [0.18913724]
 [0.93200606]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Step 1 Loss: tf.Tensor(3.479525, shape=(), dtype=float32)


INFO:absl:generating... batch 2


...

tf.Tensor(
[[0.00332795]
 [0.00292921]
 [0.0048343 ]
 [0.00288828]
 [0.99373996]
 [0.9958954 ]
 [0.9789468 ]
 [0.9967722 ]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Step 7 Loss: tf.Tensor(0.048908263, shape=(), dtype=float32)


INFO:absl:generating... batch 8


tf.Tensor(
[[0.00347727]
 [0.0019658 ]
 [0.00131783]
 [0.00226332]
 [0.9942227 ]
 [0.9992489 ]
 [0.99911827]
 [0.99545103]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Step 8 Loss: tf.Tensor(0.021021552, shape=(), dtype=float32)


INFO:absl:generating... batch 9


tf.Tensor(
[[1.3106562e-03]
 [9.8069850e-04]
 [2.6087391e-03]
 [7.8932499e-04]
 [9.9905509e-01]
 [9.9697351e-01]
 [9.9864846e-01]
 [9.9749184e-01]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
Step 9 Loss: tf.Tensor(0.013533685, shape=(), dtype=float32)


In [7]:
# predict a batch of real and a batch of fake
batch1 = next(combined_iter)
batch2 = next(combined_iter)
print(discriminator(batch1), batch1['label'])
print(discriminator(batch2), batch2['label'])

INFO:absl:generating... batch 0
INFO:absl:generating... batch 1


tf.Tensor(
[[0.7791218 ]
 [0.642511  ]
 [0.7202227 ]
 [0.7954696 ]
 [0.4309826 ]
 [0.8905689 ]
 [0.35913855]
 [0.59320056]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)
tf.Tensor(
[[0.5767123 ]
 [0.6641857 ]
 [0.5448191 ]
 [0.4997734 ]
 [0.6979213 ]
 [0.5774372 ]
 [0.41561309]
 [0.9954685 ]], shape=(8, 1), dtype=float32) tf.Tensor(
[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]], shape=(8, 1), dtype=float32)


### GAN

In [7]:
class GAN(ddsp.training.models.Model):
    def __init__(self, gen, discriminator, losses=[binary_crossentropy()]):
        super().__init__(name='gan_model', losses=losses)
        self.gen = gen
        self.discriminator = discriminator
        
    def call(self, inputs, batch_size=8): #inputs are NONE
        # label and training param don't matter here - we add our own label and loss
        generated = self.gen.generate_batch(batch_size=batch_size)
        classification = self.discriminator(generated)
        label = tf.convert_to_tensor([1]) #trying to trick the frozen discriminator 
        self.add_losses(label, classification)
        
        return classification
        

In [14]:
gan_model = GAN(gen,discriminator)
# gan_model(None,batch_size=batch_size)

In [9]:
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

In [12]:
gan_model.fit(1,1)

RuntimeError: in converted code:

    <ipython-input-7-a48792ba4a5c>:10 call  *
        classification = self.discriminator(generated)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:773 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /Users/trippgordon/Desktop/Code/sonic/NoisyGAN/core/discriminator.py:47 call  *
        self.add_losses(label, classification)
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/ddsp/training/models.py:62 add_losses  *
        self.add_loss(loss_obj(audio, audio_gen))
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer.py:1061 add_loss
        base_layer_utils.check_graph_consistency(loss, method='add_loss')
    /Users/trippgordon/miniconda3/envs/ddsp/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/base_layer_utils.py:613 check_graph_consistency
        correc

In [69]:
def train_generator_loop(gan_model, opt, iters=10,batch_size=8):
    # freeze weights
    discriminator.trainable = False
    
    # discriminator weights are frozen - just being used as loss function
    for i in range(iters):
        # train_step
        grad_clip_norm = 3.0
        with tf.GradientTape() as tape:
            pred = gan_model(None,batch_size=batch_size)
            print(pred)
            total_loss = tf.reduce_sum(gan_model.losses)
            print("Step " + str(i) + " Loss: " + str(total_loss))
        grads = tape.gradient(total_loss, gan_model.trainable_variables)
        grads, _ = tf.clip_by_global_norm(grads, grad_clip_norm)
        opt.apply_gradients(zip(grads, gan_model.trainable_variables))

In [83]:
train_generator_loop(gan_model,opt,iters=5,batch_size=batch_size)

tf.Tensor(
[[0.22240552]
 [0.29115674]
 [0.24424721]
 [0.28603742]
 [0.2317882 ]
 [0.28511062]
 [0.20704725]
 [0.31260324]], shape=(8, 1), dtype=float32)
Step 0 Loss: tf.Tensor(13.271577, shape=(), dtype=float32)
tf.Tensor(
[[0.44365394]
 [0.41424534]
 [0.4681867 ]
 [0.4949361 ]
 [0.5309448 ]
 [0.43271983]
 [0.46563256]
 [0.49377242]], shape=(8, 1), dtype=float32)
Step 1 Loss: tf.Tensor(11.16415, shape=(), dtype=float32)
tf.Tensor(
[[0.37190312]
 [0.40478504]
 [0.4288041 ]
 [0.37026948]
 [0.40472698]
 [0.3566368 ]
 [0.3875132 ]
 [0.3782742 ]], shape=(8, 1), dtype=float32)
Step 2 Loss: tf.Tensor(11.521163, shape=(), dtype=float32)
tf.Tensor(
[[0.46543813]
 [0.43919566]
 [0.46616435]
 [0.5195645 ]
 [0.46771052]
 [0.45212594]
 [0.49280515]
 [0.48270252]], shape=(8, 1), dtype=float32)
Step 3 Loss: tf.Tensor(11.130957, shape=(), dtype=float32)
tf.Tensor(
[[0.4796459 ]
 [0.51516396]
 [0.47659773]
 [0.47755668]
 [0.4395507 ]
 [0.48955938]
 [0.48402855]
 [0.48897177]], shape=(8, 1), dtype=floa

In [84]:
play(gen.generate()['audio'])

In [ ]:
#full train loop
# train_discriminator_loop(discriminator, opt, combined_iter, iters=10)

### Test weights to see if updates are happening when they should be

In [75]:
gen.weights #4.7176019e-10,  1.7397384e-10,  7.5776972e-12

[<tf.Variable 'dense_1/kernel:0' shape=(100, 2000) dtype=float32, numpy=
 array([[-0.01118855, -0.04410762, -0.00040803, ...,  0.00340417,
         -0.00311773,  0.0069016 ],
        [ 0.01804968,  0.01145471, -0.04795916, ..., -0.0041727 ,
         -0.0302236 , -0.0354804 ],
        [ 0.01869192, -0.001838  , -0.02589838, ..., -0.00942198,
          0.04158749, -0.00378041],
        ...,
        [-0.01361629, -0.04520887, -0.00524813, ..., -0.04012864,
         -0.02602137,  0.02651953],
        [-0.04238617, -0.04353556,  0.04332904, ...,  0.04000583,
         -0.03848475,  0.04565734],
        [-0.00182078, -0.03508902,  0.01600931, ..., -0.02745702,
         -0.01054335,  0.04502236]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(2000,) dtype=float32, numpy=
 array([ 1.7837894e-03, -4.0167486e-03,  1.8578753e-03, ...,
         5.8037689e-04,  1.0781414e-06, -2.0186148e-04], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/gamma:0' shape=(2000,) dtype=float32, numpy=


In [74]:
discriminator.weights # 0.9885848 , 0.98497814, 0.99335194,

[<tf.Variable 'discriminator/mfcc_time_distrbuted_rnn_encoder/normalize/scale:0' shape=(1, 1, 1, 30) dtype=float32, numpy=
 array([[[[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
           1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]]],
       dtype=float32)>,
 <tf.Variable 'discriminator/mfcc_time_distrbuted_rnn_encoder/normalize/shift:0' shape=(1, 1, 1, 30) dtype=float32, numpy=
 array([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]],
       dtype=float32)>,
 <tf.Variable 'discriminator/mfcc_time_distrbuted_rnn_encoder/gru/kernel:0' shape=(30, 1536) dtype=float32, numpy=
 array([[-0.01209034,  0.02841852, -0.02640772, ..., -0.05727458,
          0.05817929,  0.04381523],
        [-0.0245156 ,  0.01383137,  0.00931423, ...,  0.04555096,
          0.05003124, -0.03552907],
        [ 0.01280572, -0.0320526 ,  0.00983945, ...,  0.05313539,
          0.01385737,  0.0513